In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Device: {device}")
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version:", torch.version.cuda)
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

Device: cuda
PyTorch Version: 2.6.0+cu124
CUDA Available: True
CUDA Version: 12.4
GPU Name: NVIDIA GeForce RTX 4070 Laptop GPU


In [6]:
train_df = pd.read_csv('data/train.tsv', sep='\t', header=None)
test_df = pd.read_csv('data/test.tsv', sep='\t', header=None)
imdb_df = pd.read_csv('data/IMDB-Dataset.csv')

train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

display(train_df.shape, val_df.shape, test_df.shape)

(5536, 2)

(1384, 2)

(1821, 2)

In [8]:
model = nn.Sequential(
    nn.Linear(2, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(64, 2),
    nn.ReLU(),
    nn.Dropout(0.3)
).to(device)

summary(model, (2,))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [2]                       --
├─Linear: 1-1                            [512]                     1,536
├─ReLU: 1-2                              [512]                     --
├─Dropout: 1-3                           [512]                     --
├─Linear: 1-4                            [256]                     131,328
├─ReLU: 1-5                              [256]                     --
├─Dropout: 1-6                           [256]                     --
├─Linear: 1-7                            [128]                     32,896
├─ReLU: 1-8                              [128]                     --
├─Dropout: 1-9                           [128]                     --
├─Linear: 1-10                           [64]                      8,256
├─ReLU: 1-11                             [64]                      --
├─Dropout: 1-12                          [64]                      --
